In [ ]:
import os
import pandas as pd
import hashlib
from typing import List
from catboost import CatBoostClassifier
from fastapi import FastAPI
from datetime import datetime
from sqlalchemy import create_engine
from schema import PostGet, Response

# ==================== КОНФИГУРАЦИЯ A/B ЭКСПЕРИМЕНТА ====================
SALT = "recommender_salt_2025"  # Соль для хэширования
CONTROL_GROUP_RATIO = 0.5  # 50% пользователей в контрольной группе
TEST_GROUP_RATIO = 0.5     # 50% пользователей в тестовой группе

# ==================== ИНИЦИАЛИЗАЦИЯ ПРИЛОЖЕНИЯ ====================
app = FastAPI()

def batch_load_sql(query: str) -> pd.DataFrame:
    """
    Загрузка больших объемов данных из PostgreSQL с потоковой обработкой.
    """
    engine = create_engine(
        "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
    )
    conn = engine.connect().execution_options(stream_results=True)
    
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn, chunksize=200000):
        chunks.append(chunk_dataframe)
    
    conn.close()
    return pd.concat(chunks, ignore_index=True)


def get_model_path(model_name: str, path: str) -> str:
    """
    Определение пути к модели в зависимости от окружения и имени модели.
    """
    if os.environ.get("IS_LMS") == "1":
        return f'/workdir/user_input/{model_name}'
    else:
        return path


def load_models() -> dict:
    """
    Загрузка обеих моделей: контрольной и тестовой.
    """
    # Загрузка контрольной модели (без нейросетей)
    model_control_path = get_model_path("model_control", "recommender_model_v_exp.cbm")
    model_control = CatBoostClassifier()
    model_control.load_model(model_control_path)
    
    # Загрузка тестовой модели (с нейросетями)
    model_test_path = get_model_path("model_test", "recommender_model_v_exp_neural.cbm")
    model_test = CatBoostClassifier()
    model_test.load_model(model_test_path)
    
    return {
        'control': model_control,
        'test': model_test
    }


def load_features() -> dict:
    """
    Загрузка всех необходимых данных для рекомендаций.
    """
    # Загрузка информации о лайкнутых постах
    liked_posts_query = """
        SELECT DISTINCT post_id, user_id
        FROM feed_data
        WHERE action = 'like'
    """
    liked_posts = batch_load_sql(liked_posts_query)

    # Загрузка признаков постов для контрольной модели (без нейросетей)
    posts_features_control = pd.read_sql(
        "SELECT * FROM k_m_exp_38_post_features_lesson_22",
        con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
            "postgres.lab.karpov.courses:6432/startml"
    )

    # Загрузка признаков постов для тестовой модели (с нейросетями)
    posts_features_test = pd.read_sql(
        "SELECT * FROM k_m_exp_neural_38_post_features_lesson_22",
        con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
            "postgres.lab.karpov.courses:6432/startml"
    )

    # Загрузка данных пользователей
    user_features = pd.read_sql(
        "SELECT * FROM user_data",
        con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
            "postgres.lab.karpov.courses:6432/startml"
    )

    return {
        'liked_posts': liked_posts,
        'posts_features_control': posts_features_control,
        'posts_features_test': posts_features_test,
        'user_features': user_features
    }


def get_exp_group(user_id: int) -> str:
    """
    Определение экспериментальной группы пользователя на основе хэша user_id.
    
    Args:
        user_id: ID пользователя
        
    Returns:
        str: 'control' или 'test'
    """
    # Создаем хэш от user_id с солью
    hash_str = hashlib.md5(f"{user_id}_{SALT}".encode()).hexdigest()
    
    # Преобразуем хэш в число от 0 до 1
    hash_ratio = int(hash_str[:8], 16) / 0xFFFFFFFF
    
    # Определяем группу на основе заданных пропорций
    if hash_ratio < CONTROL_GROUP_RATIO:
        return "control"
    else:
        return "test"


# ==================== ЗАГРУЗКА МОДЕЛЕЙ И ДАННЫХ ПРИ СТАРТЕ ====================
print("Загрузка моделей...")
models = load_models()
model_control = models['control']
model_test = models['test']

print("Загрузка признаков...")
features = load_features()

# Определяем ожидаемые колонки для каждой модели
EXPECTED_FEATURES_CONTROL = [
    'topic', 'TotalTfIdf', 'MaxTfIdf', 'MeanTfIdf', 'TextCluster',
    'DistanceToCluster_1', 'DistanceToCluster_2', 'DistanceToCluster_3',
    'DistanceToCluster_4', 'DistanceToCluster_5', 'DistanceToCluster_6', 
    'DistanceToCluster_7', 'DistanceToCluster_8', 'DistanceToCluster_9',
    'DistanceToCluster_10', 'DistanceToCluster_11', 'DistanceToCluster_12',
    'DistanceToCluster_13', 'DistanceToCluster_14', 'DistanceToCluster_15',
    'gender', 'age', 'country', 'city', 'exp_group', 'os', 'source', 'hour', 'month'
]

EXPECTED_FEATURES_TEST = [
    'topic', 'EmbeddingNorm', 'EmbeddingMean', 'EmbeddingStd', 'TextCluster',
    'DistanceToCluster_1', 'DistanceToCluster_2', 'DistanceToCluster_3',
    'DistanceToCluster_4', 'DistanceToCluster_5', 'DistanceToCluster_6', 
    'DistanceToCluster_7', 'DistanceToCluster_8', 'DistanceToCluster_9',
    'DistanceToCluster_10', 'DistanceToCluster_11', 'DistanceToCluster_12',
    'DistanceToCluster_13', 'DistanceToCluster_14', 'DistanceToCluster_15',
    'PC_1', 'PC_2', 'PC_3', 'PC_4', 'PC_5', 'PC_6', 'PC_7', 'PC_8', 'PC_9', 'PC_10',
    'PC_11', 'PC_12', 'PC_13', 'PC_14', 'PC_15', 'PC_16', 'PC_17', 'PC_18', 'PC_19', 'PC_20',
    'gender', 'age', 'country', 'city', 'exp_group', 'os', 'source', 'hour', 'month'
]

# Категориальные признаки (одинаковые для обеих моделей)
CATEGORICAL_FEATURES = [
    'topic', 'TextCluster', 'gender', 'country', 
    'city', 'exp_group', 'hour', 'month', 'os', 'source'
]


def get_recommendations_control(user_id: int, time: datetime, limit: int) -> List[PostGet]:
    """
    Генерация рекомендаций с использованием контрольной модели (без нейросетей).
    """
    print(f"Применяем контрольную модель для пользователя {user_id}")
    
    # Получение признаков пользователя
    user_data = features['user_features'].loc[features['user_features'].user_id == user_id]
    if user_data.empty:
        print(f"Пользователь {user_id} не найден")
        return []
    
    user_data = user_data.drop('user_id', axis=1)

    # Поиск колонки с идентификатором поста
    post_id_column = None
    possible_post_id_names = ['post_id', 'id', 'post_id', 'postid']
    
    for col_name in possible_post_id_names:
        if col_name in features['posts_features_control'].columns:
            post_id_column = col_name
            break
    
    if post_id_column is None:
        print("Ошибка: не найдена колонка с идентификатором поста в контрольных данных")
        return []

    # Подготовка признаков постов для контрольной модели
    available_features = [col for col in EXPECTED_FEATURES_CONTROL if col in features['posts_features_control'].columns]
    
    posts_data = features['posts_features_control'][available_features].copy()
    posts_data[post_id_column] = features['posts_features_control'][post_id_column]
    
    # Данные для ответа
    text_column = 'text'
    topic_column = 'topic'
    post_content = features['posts_features_control'][[post_id_column, text_column, topic_column]].copy()

    # Создание объединенного датафрейма
    user_features_dict = dict(zip(user_data.columns, user_data.values[0]))
    user_posts_data = posts_data.assign(**user_features_dict)
    user_posts_data = user_posts_data.set_index(post_id_column)

    # Добавление временных признаков
    user_posts_data['hour'] = time.hour
    user_posts_data['month'] = time.month

    # Предсказание
    try:
        predictions = model_control.predict_proba(user_posts_data)[:, 1]
        user_posts_data['predicts'] = predictions
    except Exception as e:
        print(f"Ошибка при предсказании контрольной моделью: {e}")
        return []

    # Фильтрация лайкнутых постов
    user_liked_posts = features['liked_posts'][features['liked_posts'].user_id == user_id][post_id_column].values
    filtered_posts = user_posts_data[~user_posts_data.index.isin(user_liked_posts)]

    # Выбор топ-N постов
    recommended_post_ids = filtered_posts.sort_values('predicts')[-limit:].index

    # Формирование ответа
    recommendations = []
    for post_id in recommended_post_ids:
        post_info = post_content[post_content[post_id_column] == post_id]
        if not post_info.empty:
            recommendations.append(
                PostGet(**{
                    "id": int(post_id),
                    "text": post_info[text_column].values[0],
                    "topic": post_info[topic_column].values[0]
                })
            )
    
    print(f"Контрольная модель сгенерировала {len(recommendations)} рекомендаций для пользователя {user_id}")
    return recommendations


def get_recommendations_test(user_id: int, time: datetime, limit: int) -> List[PostGet]:
    """
    Генерация рекомендаций с использованием тестовой модели (с нейросетями).
    """
    print(f"Применяем тестовую модель для пользователя {user_id}")
    
    # Получение признаков пользователя
    user_data = features['user_features'].loc[features['user_features'].user_id == user_id]
    if user_data.empty:
        print(f"Пользователь {user_id} не найден")
        return []
    
    user_data = user_data.drop('user_id', axis=1)

    # Поиск колонки с идентификатором поста
    post_id_column = None
    possible_post_id_names = ['post_id', 'id', 'post_id', 'postid']
    
    for col_name in possible_post_id_names:
        if col_name in features['posts_features_test'].columns:
            post_id_column = col_name
            break
    
    if post_id_column is None:
        print("Ошибка: не найдена колонка с идентификатором поста в тестовых данных")
        return []

    # Подготовка признаков постов для тестовой модели
    available_features = [col for col in EXPECTED_FEATURES_TEST if col in features['posts_features_test'].columns]
    
    posts_data = features['posts_features_test'][available_features].copy()
    posts_data[post_id_column] = features['posts_features_test'][post_id_column]
    
    # Данные для ответа
    text_column = 'text'
    topic_column = 'topic'
    post_content = features['posts_features_test'][[post_id_column, text_column, topic_column]].copy()

    # Создание объединенного датафрейма
    user_features_dict = dict(zip(user_data.columns, user_data.values[0]))
    user_posts_data = posts_data.assign(**user_features_dict)
    user_posts_data = user_posts_data.set_index(post_id_column)

    # Добавление временных признаков
    user_posts_data['hour'] = time.hour
    user_posts_data['month'] = time.month

    # Предсказание
    try:
        predictions = model_test.predict_proba(user_posts_data)[:, 1]
        user_posts_data['predicts'] = predictions
    except Exception as e:
        print(f"Ошибка при предсказании тестовой моделью: {e}")
        return []

    # Фильтрация лайкнутых постов
    user_liked_posts = features['liked_posts'][features['liked_posts'].user_id == user_id][post_id_column].values
    filtered_posts = user_posts_data[~user_posts_data.index.isin(user_liked_posts)]

    # Выбор топ-N постов
    recommended_post_ids = filtered_posts.sort_values('predicts')[-limit:].index

    # Формирование ответа
    recommendations = []
    for post_id in recommended_post_ids:
        post_info = post_content[post_content[post_id_column] == post_id]
        if not post_info.empty:
            recommendations.append(
                PostGet(**{
                    "id": int(post_id),
                    "text": post_info[text_column].values[0],
                    "topic": post_info[topic_column].values[0]
                })
            )
    
    print(f"Тестовая модель сгенерировала {len(recommendations)} рекомендаций для пользователя {user_id}")
    return recommendations


def get_recommended_feed(user_id: int, time: datetime, limit: int) -> Response:
    """
    Основная функция для генерации рекомендаций с A/B тестированием.
    
    Returns:
        Response: Объект с группой и рекомендациями
    """
    # Определяем группу пользователя
    exp_group = get_exp_group(user_id)
    print(f"Пользователь {user_id} определен в группу: {exp_group}")
    
    # Выбираем соответствующую модель
    if exp_group == "control":
        recommendations = get_recommendations_control(user_id, time, limit)
    elif exp_group == "test":
        recommendations = get_recommendations_test(user_id, time, limit)
    else:
        raise ValueError(f"Неизвестная группа: {exp_group}")
    
    # Возвращаем ответ с указанием группы
    return Response(
        exp_group=exp_group,
        recommendations=recommendations
    )


# ==================== API ENDPOINT ====================
@app.get("/post/recommendations/", response_model=Response)
def recommended_posts(
    id: int, 
    time: datetime, 
    limit: int = 10
) -> Response:
    """
    API endpoint для получения рекомендаций постов с A/B тестированием.
    
    Returns:
        Response: Объект с экспериментальной группой и рекомендациями
    """
    return get_recommended_feed(id, time, limit)